In [1]:
from process_bigraph import ProcessTypes

CORE = ProcessTypes()

In [2]:
import warnings
from basico import * 
from process_bigraph import Process


warnings.filterwarnings("ignore", category=FutureWarning)


class ODECopasi(Process):
    config_schema = {
        'model_file': 'string'
    }

    def __init__(self, config=None, core=CORE):
        super().__init__(config, core)
        self.model = load_model(self.config['model_file'])
        self.reaction_names = get_reactions(model=self.model).index.tolist()
        self.species_names = get_species(model=self.model).index.tolist()

    def initial_state(self):
        initial_concentrations = {
            species_name: get_species(species_name, model=self.model).initial_concentration[0]
            for species_name in self.species_names
        }

        initial_derivatives = {
            rxn_id: get_reactions(rxn_id, model=self.model).flux[0]
            for rxn_id in self.reaction_names
        }

        return {
            'species_concentrations': initial_concentrations,
            'reaction_fluxes': initial_derivatives,
            'time': 0.0
        }

    def inputs(self):
        concentrations_type = {
            name: 'float' for name in self.species_names
        }
        return {
            'species_concentrations': concentrations_type,
            'time': 'float'
        }

    def outputs(self):
        concentrations_type = {
            name: 'float' for name in self.species_names
        }

        reaction_fluxes_type = {
            reaction_name: 'float' for reaction_name in self.reaction_names
        }

        return {
            'species_concentrations': concentrations_type,
            'reaction_fluxes': reaction_fluxes_type,
            'time': 'float'
        }

    def update(self, inputs, interval):
        for cat_id, value in inputs['species_concentrations'].items():
            set_type = 'concentration'
            species_config = {
                'name': cat_id,
                'model': self.model,
                set_type: value
            }
            set_species(**species_config)

        # run model for "interval" length; we only want the state at the end
        tc = run_time_course(
            start_time=inputs['time'],
            duration=interval,
            update_model=True,
            model=self.model
        )

        results = {'time': interval}
        results['species_concentrations'] = {
            mol_id: float(get_species(
                name=mol_id,
                exact=True,
                model=self.model
            ).concentration[0])
            for mol_id in self.species_names
        }

        results['reaction_fluxes'] = {
            rxn_id: float(get_reactions(
                name=rxn_id,
                model=self.model
            ).flux[0])
            for rxn_id in self.reaction_names
        }

        return results


CORE.process_registry.register('ode-copasi', ODECopasi)

In [3]:
import logging
import os
from pathlib import Path

import cobra
from cobra.io import read_sbml_model
from process_bigraph import Process, Composite
import numpy as np


logging.getLogger('cobra').setLevel(logging.ERROR)


class FBA(Process):
    config_schema = {
        'model_file': 'string',
        'objective': {
            'domain': 'string',  # either protein or mrna
            'name': 'string',  # specific to the model: i.e., LacI
            'scaling_factor': 'float'
        }
    }

    def __init__(self, config=None, core=CORE):
        super().__init__(config, core)

        # create model
        model_file = self.config['model_file']
        data_dir = Path(os.path.dirname(model_file))
        path = data_dir / model_file.split('/')[-1]
        self.model = read_sbml_model(str(path.resolve()))

        # parse objective
        self.objective_domain = self.config['objective']['domain']
        self.objective_name = self.config['objective']['name']
        self.scaling_factor = self.config['objective'].get('scaling_factor', 10)

        # set objectives
        self.model.objective = {
            self.model.reactions.get_by_id(reaction.id): np.random.random()  # TODO: make this more realistic
            for reaction in self.model.reactions
        }

        # set even bounds
        for reaction in self.model.reactions:
            rand_bound = np.random.random()
            self.model.reactions.get_by_id(reaction.id).lower_bound = -rand_bound  # TODO: What to do here?
            self.model.reactions.get_by_id(reaction.id).upper_bound = rand_bound

    def initial_state(self):
        initial_fluxes = {}
        initial_solution = self.model.optimize()
        if initial_solution.status == 'optimal':
            initial_fluxes = {
                reaction.name: reaction.flux
                for reaction in self.model.reactions
            }

        return {'fluxes': initial_fluxes}

    def inputs(self):
        return {'reaction_fluxes': 'tree[float]'}

    def outputs(self):
        return {'fluxes': 'tree[float]'}

    def update(self, state, interval):
        for reaction_name, reaction_flux in state['reaction_fluxes'].items():
            for reaction in self.model.reactions:
                if reaction.name == reaction_name:
                    # 1. reset objective weights according to reaction fluxes directly
                    self.model.objective = {
                        self.model.reactions.get_by_id(reaction.id): reaction_flux
                    }

                    # 2. set lower bound with scaling factor and reaction fluxes
                    self.model.reactions.get_by_id(reaction.id).lower_bound = -self.scaling_factor * abs(reaction_flux)  # / (5 + abs(reaction_flux))

        # 3. solve for fluxes
        output_state = {}
        solution = self.model.optimize()
        if solution.status == "optimal":
            data = dict(zip(
                list(state['reaction_fluxes'].keys()),
                list(solution.fluxes.to_dict().values())
            ))
            output_state['fluxes'] = data

            # TODO: do we want to instead scale by input flux?
            # for reaction in self.model.reactions:
            #     flux = solution.fluxes[reaction.id]
            #     for reaction_name, reaction_flux in state['reaction_fluxes'].items():
            #         if reaction.name == reaction_name:
            #             fluxes[reaction.name] = flux * reaction_flux

        return output_state


CORE.process_registry.register('fba', FBA)

In [4]:
fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'
fp2 = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Beard2005_Mitochondrial_Respiration.xml'
fp3 = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Mitchell2013/BIOMD0000000498_url.xml'


doc = {
        'ode': {
            '_type': 'process',
            'address': f'local:ode-copasi',
            'config': {
                'model_file': fp3
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store']
            },
            'outputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_fluxes': ['reaction_fluxes_store']
            }
        },
        'fba': {
            '_type': 'process',
            'address': f'local:fba',
            'config': {
                'model_file': fp3,
            },
            'inputs': {
                'reaction_fluxes': ['reaction_fluxes_store']
            },
            'outputs': {
                'fluxes': ['fluxes_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'species_concentrations': 'tree[float]',
                    'reaction_fluxes': 'tree[float]',
                    'fluxes': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_fluxes': ['reaction_fluxes_store'],
                'fluxes': ['fluxes_store']
            }
        }
    }

In [5]:
composition = Composite(config={'state': doc}, core=CORE)

# save initial state 
composition.save(filename='dfba_initial.json', outdir='.')

No objective coefficients in model. Unclear what should be optimized


Created new file: ./dfba_initial.json


In [6]:
composition.run(200)

# save final state
composition.save(filename='dfba_final.json', outdir='.')

# get result from emitter
result = composition.gather_results()[('emitter',)]

Created new file: ./dfba_final.json


In [9]:
result[:2]

[{'time': 0.0,
  'species_concentrations': {'Hamp': 5e-09,
   'Fe-FT': 0.0,
   'FT': 0.0,
   'FT1': 0.0,
   'HO-1': 3.56e-11,
   'Heme': 1e-09,
   'LIP': 1.3e-06,
   'Fpn': 1e-09,
   'IRP': 1.16e-06,
   'Tf-Fe_intercell': 5e-06,
   'TfR': 4e-07,
   'Tf-Fe-TfR1': 0.0,
   'HFE': 2e-07,
   'HFE-TfR': 0.0,
   'Tf-Fe-TfR2': 0.0,
   '2(Tf-Fe)-TfR1': 0.0,
   '2HFE-TfR': 0.0,
   '2HFE-TfR2': 0.0,
   '2(Tf-Fe)-TfR2': 0.0,
   'TfR2': 0.0,
   'Heme_intercell': 1e-07},
  'reaction_fluxes': {'Fpn Export': 8.663778740419861e-10,
   'TfR1 expression': 3.2222222222222225e-12,
   'TfR1 degradation': 3.3479999999999995e-12,
   'Ferroportin Expression': 8.116883116883118e-10,
   'IRP expresion': 1.7391304347826085e-11,
   'IRP degradation': 1.85252e-11,
   'Fpn degradation': 1.1575e-13,
   'HFE degradation': 1.2835999999999999e-11,
   'HFE expression': 2.3469e-11,
   'TfR2 expression': 3e-11,
   'TfR2 degradation': 3.2000000000000006e-11,
   'Hepcidin expression': 0.0,
   'Hepcidin degradation': 2.799999

In [10]:
import json 

with open('dfba_final.json', 'r') as f:
    checkpoint = json.load(f)
    
composition2 = Composite(config={'state': checkpoint}, core=CORE)

No objective coefficients in model. Unclear what should be optimized


In [11]:
composition2.state

{'state': {'global_time': '200.0',
  'ode': {'_type': 'process',
   'address': 'local:ode-copasi',
   'config': {'model_file': '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Mitchell2013/BIOMD0000000498_url.xml'},
   'inputs': {'time': ['time_store'],
    'species_concentrations': ['species_concentrations_store']},
   'outputs': {'time': ['time_store'],
    'species_concentrations': ['species_concentrations_store'],
    'reaction_fluxes': ['reaction_fluxes_store']},
   'interval': 1.0,
   '_inputs': {'species_concentrations': {'Hamp': 'float',
     'Fe-FT': 'float',
     'FT': 'float',
     'FT1': 'float',
     'HO-1': 'float',
     'Heme': 'float',
     'LIP': 'float',
     'Fpn': 'float',
     'IRP': 'float',
     'Tf-Fe_intercell': 'float',
     'TfR': 'float',
     'Tf-Fe-TfR1': 'float',
     'HFE': 'float',
     'HFE-TfR': 'float',
     'Tf-Fe-TfR2': 'float',
     '2(Tf-Fe)-TfR1': 'float',
     '2HFE-TfR': 'float',
     '2HFE-TfR2': 'flo